# API ROCrate Tests

In [2]:
import requests
import json
from requests_toolbelt.multipart.encoder import MultipartEncoder

root_url = "http://localhost:8080/"

## Create users

In [4]:
user1_data = {
    "@id": "ark:99999/test-user1",
    "name": "Test User1",
    "type": "Person",
    "email": "testuser1@example.org",
    "password": "test1",
    "organizations": [],
    "projects": [],
    "datasets": [],
    "rocrates": [],
    "software": [],
    "computations": [],
    "evidencegraphs": []
}
user1_create = requests.post(root_url + "user", json=user1_data)
user1_create.json()


{'error': 'document already exists'}

## Create Organization

In [6]:
org_data = {
    "@id": "ark:99999/test-org",
    "name": "test organization",
    "@type": "Organization",
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    }
}

In [7]:
import base64
encoded_credentials = base64.b64encode(
    bytes(f"{user1_data.get('email')}:{user1_data.get('password')}", "utf-8"))

owner_headers = {"Authorization" : f"Basic {str(encoded_credentials, 'utf-8')}" }

organization_create = requests.post(root_url + "organization", data=json.dumps(org_data), headers=owner_headers)
organization_create.json()

{'created': {'@id': 'ark:99999/test-org', '@type': 'Organization'}}

## Create Project

In [9]:
project_data = {
    "@id": "ark:99999/test-org/test-proj",
    "name": "Test Project",
    "@type": "Project",
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    },
    "memberOf": {
        "@id": org_data['@id'],
        "name": org_data['name'],
        "@type": org_data['@type']
    }
}
project_create = requests.post(root_url + "project", json=project_data, headers=owner_headers)
project_create.json()

{'created': {'@id': 'ark:99999/test-org/test-proj', '@type': 'Project'}}

## Create ROCrate

In [11]:
rocrate_data = {
    "@id": "ark:99999/test-org/test-proj/test-rocrate",
    "@type": "evi:ROCrate",
    "name": "Test ROCrate",
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    }    
}
rocrate_create = requests.post(root_url + f"rocrate/", data=json.dumps(rocrate_data))
rocrate_create.json()

{'created': {'@id': 'ark:99999/test-org/test-proj/test-rocrate',
  '@type': 'evi:ROCrate'}}

## List ROCrates

In [16]:
rocrate_list = requests.get(root_url + f"rocrate")
rocrate_list.json()

{'rocrates': [{'@id': 'ark:99999/test-org/test-proj/test-rocrate',
   '@type': 'evi:ROCrate',
   'name': 'Test ROCrate'}]}

In [17]:
distribution_data = {
    "@id": "ark:99999/test-org/test-proj/test-rocrate/guid-test-rocrate.zip",
    "@type": "DataDownload",
    "name": "Test ROCrate",
    "encodingFormat": ".zip",
    "encodesCreativeWork": {
        "@id": rocrate_data['@id'],
        "@type": rocrate_data['@type'],
        "name": rocrate_data['name']
    },
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    }
}

In [20]:
mp_encoder = MultipartEncoder(
    fields={
        'download': json.dumps(distribution_data),
        # plain file object, no filename or mime type produces a
        # Content-Disposition header with just the part name
        'file': ('test-rocrate', open('/home/sadnan/PycharmProjects/mds_python/tests/guid_test_rocrate.zip', 'rb'), 'application/zip '),
    }
)

# upload a software to minio object store
# dataset_upload = {'download': str(download_data), 'file': open("tests/test-data.csv", "rb")}

data_download_upload = requests.post(root_url + f"register",                              
    data=mp_encoder,  # The MultipartEncoder is posted as data, don't use files=...!
    # The MultipartEncoder provides the content-type header with the boundary:
    headers={'Content-Type': mp_encoder.content_type}
                )
                    
data_download_upload.json()

{'created': {'@id': 'ark:99999/test-org/test-proj/test-rocrate/guid-test-rocrate.zip',
  '@type': 'Download',
  'name': 'Test ROCrate'}}